<div align="center">
    <img src='../images/hw3.png' alt='hw2-log'/>
</div>

## Import Libraries

In [226]:
# Scientific
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge

from collections import deque
import warnings
warnings.filterwarnings('ignore')

## Download Dataset

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-03 11:14:04--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M   315KB/s    in 4.6s    

2023-10-03 11:14:11 (315 KB/s) - ‘data.csv’ saved [1475504/1475504]



## Load Dataset

In [132]:
data = pd.read_csv('../dataset/data.csv')

In [133]:
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


## Feature Selection

In [134]:
columns = ['Make', 'Model', 'Year','Engine HP','Engine Cylinders',
           'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']

In [135]:
filterd_data = data[columns].copy()

## Data Prepration

In [136]:
filterd_data.columns = filterd_data.columns.str.replace(' ', '_').str.lower()

In [137]:
filterd_data.isna().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [138]:
filterd_data.fillna(0, inplace=True)

In [139]:
filterd_data.isna().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [140]:
filterd_data.rename(columns = {'msrp':'price'}, inplace=True)

## Question 1: What is the most frequent observation (mode) for the column ```transmission_type```?

In [141]:
filterd_data.transmission_type.value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

As we can see AUTOMATIC: 8266 is the mode

### Question 2: Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

### What are the two features that have the biggest correlation in this dataset?

In [142]:
filterd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11914 non-null  float64
 4   engine_cylinders   11914 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   price              11914 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 930.9+ KB


In [143]:
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']

In [144]:
filterd_data[numerical].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


## make price binary

In [145]:

filterd_data['above_average'] = filterd_data.price.apply(lambda x: 1 if x > filterd_data.price.mean() else 0 )

## Split Data

In [146]:
X = filterd_data[filterd_data.columns[:-1]]
y = filterd_data[filterd_data.columns[-1]]

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    random_state=42,
                                                    test_size=0.20)

In [148]:
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    stratify=y_train,
                                                    random_state=42,
                                                    test_size=0.20)

## Question 3: 
- Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the lowest mutual information score?



In [222]:
lowest_mutul_info = float('inf')
for column in categorical:
    mutual_info = round(mutual_info_score(X_train[column], y_train), 2)
    print(f'mutual information between above_average & {column}: {mutual_info}')
    
    if mutual_info < lowest_mutul_info:
        lowest_mutul_info = mutual_info
        lowest_column = column
        
print("_______________________________________")
print(f'The column with lowest mutual information is: {lowest_column}, mutual info: {lowest_mutul_info}')    

mutual information between above_average & make: 0.23
mutual information between above_average & model: 0.46
mutual information between above_average & transmission_type: 0.02
mutual information between above_average & vehicle_style: 0.08
_______________________________________
The column with lowest mutual information is: transmission_type, mutual info: 0.02


## Question 4: 
- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

## One-Hot

In [223]:
dv = DictVectorizer(sparse=False)

train_dict = X_train[categorical + numerical].to_dict(orient='records')
X_train1 = dv.fit_transform(train_dict)

val_dict = X_val[categorical + numerical].to_dict(orient='records')
X_val1 = dv.transform(val_dict)

## Fit model

In [224]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
_ = model.fit(X_train1, y_train)

In [225]:
accuracy = model.score(X_val1, y_val)
round(accuracy, 2)

0.93

## Question 5: 
- Let's find the least useful feature using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [178]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
_ = model.fit(X_train1, y_train)
original_accuracy = model.score(X_val1, y_val)
round(original_accuracy, 2)

0.93

In [190]:
best_diff_accuracy = float('inf')
columns = filterd_data.columns[:-2]
for i in range(len(columns)):
    
    dv = DictVectorizer(sparse=False)

    train_dict = X_train[columns[1:]].to_dict(orient='records')
    X_train2 = dv.fit_transform(train_dict)

    val_dict = X_val.to_dict(orient='records')
    X_val2 = dv.transform(val_dict)
    
    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    _ = model.fit(X_train2, y_train)
    accuracy = model.score(X_val2, y_val)
    
    diff = original_accuracy - accuracy
    
    if diff < best_diff_accuracy:
        best_diff_accuracy = diff
        best_column = columns[0]
        
    print(f'Droped feature: {columns[0]}, accuracy: {accuracy}, diff: {diff}')
    
    columns = deque(columns)
    columns.rotate(-1)
    columns = list(columns)
print(f'______________________________________________')
print(f'best column: {best_column}, diff: {best_diff_accuracy}')

Droped feature: make, accuracy: 0.9339276350288411, diff: -0.0031463030938647663
Droped feature: model, accuracy: 0.908232826428946, diff: 0.02254850550603038
Droped feature: year, accuracy: 0.9449396958573676, diff: -0.014158363922391226
Droped feature: engine_hp, accuracy: 0.9260618772941793, diff: 0.004719454640797038
Droped feature: engine_cylinders, accuracy: 0.9355007865757735, diff: -0.004719454640797149
Droped feature: transmission_type, accuracy: 0.9150498164656529, diff: 0.015731515469323498
Droped feature: vehicle_style, accuracy: 0.912952281069743, diff: 0.017829050865233342
Droped feature: highway_mpg, accuracy: 0.9318300996329313, diff: -0.001048767697954922
Droped feature: city_mpg, accuracy: 0.9396958573675931, diff: -0.008914525432616727
______________________________________________
best column: year, diff: -0.014158363922391226


# Question 3 :
- For this question, we'll see how to use a linear regression model from Scikit-Learn.
- We'll need to use the original column price. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
- This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
- Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

In [206]:
new_data = filterd_data.copy()

In [207]:
new_data.drop(columns=['above_average'], inplace=True)

In [208]:
X_new = new_data[new_data.columns[:-1]]
y_new = new_data[new_data.columns[-1]]

In [214]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new,
                                                    y_new,
                                                    random_state=42,
                                                    test_size=0.20)

In [215]:
X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(X_train_new,
                                                    y_train_new,
                                                    random_state=42,
                                                    test_size=0.20)

In [216]:
dv = DictVectorizer(sparse=False)

train_dict = X_train_new.to_dict(orient='records')
X_train_new = dv.fit_transform(train_dict)

val_dict = X_val.to_dict(orient='records')
X_val_new = dv.transform(val_dict)

In [217]:
def rmse(y, y_pred):
    error =  y - y_pred
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [228]:
best_rmse = float('inf')
for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X_train_new, y_train_new)
    
    y_pred = model.predict(X_val_new)
    rmse_score = round(rmse(y_val_new, y_pred), 3)
    
    if rmse_score < best_rmse:
        best_rmse = rmse_score
        best_alpha = alpha
    print(f'alpha: {alpha}, RMSE: {rmse_score}')
    
print('________________________') 
print(f'Best RMSE: {best_rmse}, Best alpha: {alpha}')

alpha: 0, RMSE: 86515.475
alpha: 0.01, RMSE: 86515.468
alpha: 0.1, RMSE: 86515.398
alpha: 1, RMSE: 86514.698
alpha: 10, RMSE: 86507.755
________________________
Best RMSE: 86507.755, Best alpha: 10
